In [1]:
import os
import pandas as pd
import sqlite3 as sqlite

In [2]:
data = pd.read_csv("scraped.csv").query("team != 'team/'")

In [3]:
names = pd.read_csv("names.csv", encoding="utf-8")

In [4]:
sport_ents = names[~names["name"].str.startswith("Ind-")].copy()

In [5]:
graph = pd.DataFrame()
graph["A"] = data["rider"]
graph["B"] = -data["team"]

graph = graph[graph.isin(set(sport_ents["url"])).all(axis=1)]
sport_ents = sport_ents[sport_ents["url"].isin(set(graph.stack()))]

graph_sql = graph.to_numpy().tolist() + graph.to_numpy()[:,::-1].tolist()

In [6]:
path_to_db = "../docs/graph.sqlite"
try:
    try:
        os.remove(path_to_db)
    except:
        conn.close()
        del conn
        os.remove(path_to_db)
except FileNotFoundError:
    pass

In [7]:
bad_team_rider_combos = [
    (18468, -17275),
    (2665, -2305),
    (2665, -1768),
    (87656, -20911),
    (87656, -20912),
    (11500, -4699),
    (62162, -18287),
    (1492, -4887),
    (21848, -4714),
    (517, -4565),
    (517, -4566),
    (9489, -21170),
    (14157, -2195),
    (2620, -23957),
    (2620, -2968),
    (2620, -23829),
    (19440, -23611),
    (5250, -2205),
    (5250, -1212),
    (5250, -2484),
    (10509, -18082),
    (2660, -4959),
    (126466, -17812),
    (43136, -14619),
    (7151, -971),
    (10193, -3037),
    (10193, -4679),
    (10193, -4680),
    (3952, -810),
    (6710, -841),
    (58449, -18341),
    (126438, -20214),
    (126438, -17808)
]
bad_team_rider_combos = " or ".join(f"(a={i} and b ={j}) or (a={j} and b={i})" for i, j in bad_team_rider_combos)

with sqlite.connect(path_to_db) as conn:
    conn.execute(
        """CREATE TABLE sport_ents (
                    name text NOT NULL,
                    url int NOT NULL,
                    year int NOT NULL
                ) """
    )
    conn.execute("CREATE INDEX urls ON sport_ents (url) ")
    conn.executemany("INSERT INTO sport_ents VALUES (?, ?, ?)", sport_ents.to_numpy().tolist())
    conn.execute(
        """CREATE TABLE graph (
                    a int NOT NULL,
                    b int NOT NULL)"""
    )
    conn.executemany("INSERT INTO graph VALUES (?, ?)", graph_sql)
    # remove bad entries
    conn.execute(f"delete from graph where {bad_team_rider_combos}")
    conn.execute("CREATE INDEX graph_roots ON graph (a) ")
conn.close()
del conn